# Forecasting Combinations

### Loading Libraries

In [ ]:
%cd ../..

In [ ]:
# Numerical Computing
import numpy as np

# Data Manipulation
import pandas as pd

# Data Visualization
import plotly.express as px
import plotly.graph_objects as go
import plotly.io as pio

# OS & Time
import os
import time
import joblib

# Warnings, Func & Path
import warnings
import pickleshare
import missingno as msno
from pathlib import Path
from itertools import cycle
from functools import partial
from typing import List, Tuple

# PyArrow
import pyarrow as pa

# Humanize
import humanize

# Scikit-Learn
from sklearn.preprocessing import StandardScaler

# Notebook Optimizer
from tqdm.autonotebook import tqdm

# IPython - Display
from IPython.display import display, HTML

# Random
import random

In [ ]:
# Custom Libraries
from src.utils import plotting_utils
from src.utils.general import LogTime
from src.utils.data_utils import _get_32_bit_dtype 
from src.utils.ts_utils_updated import metrics_adapter, forecast_bias,mae, mase, mse

In [ ]:
# %load_ext autoreload
# %autoreload 2

In [ ]:
tqdm.pandas()

random.seed(42)

np.random.seed(42)

pio.templates.default = "plotly_white"

In [ ]:
os.makedirs("imgs/chapter_09", exist_ok=True)

preprocessed = Path.home() / "Desktop" / "data" / "london_smart_meters" / "preprocessed"

output = Path.home() / "Desktop" / "data" / "london_smart_meters" / "output"

In [ ]:
def format_plot(fig, legends=None, xlabel="Time", ylabel="Value", title="", font_size=15):
    if legends:
        names = cycle(legends)
        fig.for_each_trace(lambda t: t.update(name=next(names)))
    fig.update_layout(
        autosize=False,
        width=900,
        height=500,
        title_text=title,
        title={"x": 0.5, "xanchor": "center", "yanchor": "top"},
        titlefont={"size": 20},
        legend_title=None,
        legend=dict(
                font=dict(size=font_size),
                orientation="h",
                yanchor="bottom",
                y=0.98,
                xanchor="right",
                x=1,
            ),
            yaxis=dict(
                title_text=ylabel,
                titlefont=dict(size=font_size),
                tickfont=dict(size=font_size),
            ),
            xaxis=dict(
                title_text=xlabel,
                titlefont=dict(size=font_size),
                tickfont=dict(size=font_size),
            ),
    )
    return fig

### Reading the Test Predictions and Metrics

In [ ]:
try:
    # Reading the missing value imputed and train test split data
    train_df = pd.read_parquet(
        preprocessed / "selected_blocks_train_missing_imputed_feature_engg.parquet"
    )
    train_df = train_df.loc[:, ["timestamp", "LCLid", "energy_consumption"]].set_index(
        ["timestamp", "LCLid"]
    )
    val_df = pd.read_parquet(
        preprocessed / "selected_blocks_val_missing_imputed_feature_engg.parquet"
    )
    val_df = val_df.loc[:, ["timestamp", "LCLid", "energy_consumption"]].set_index(
        ["timestamp", "LCLid"]
    )

    train_target = train_df.reset_index().set_index("timestamp")
    # Combine train and val into new train
    train_val_target = pd.concat([train_df, val_df]).reset_index().set_index("timestamp")

    del val_df, train_df
except FileNotFoundError:
    display(HTML("""
    <div class="alert alert-block alert-warning">
    <b>Warning!</b> File not found. Please make sure you have run 01-Feature Engineering.ipynb in Chapter06
    </div>
    """))

In [ ]:
try:
    pred_test_df = pd.read_pickle(output / "ml_single_step_prediction_test_df.pkl")
    metrics_test_df = pd.read_pickle(output / "ml_single_step_metrics_test_df.pkl")
    pred_auto_stat_test_df = pd.read_pickle(
        output / "ml_single_step_prediction_auto_stationary_test_df.pkl"
    )
    metrics_auto_stat_test_df = pd.read_pickle(
        output / "ml_single_step_metrics_auto_stationary_test_df.pkl"
    )
    agg_metrics_auto_stat_test_df = pd.read_pickle(
        output / "ml_single_step_aggregate_metrics_auto_stationary_test.pkl"
    )
    pred_baselines_test_df = pd.read_pickle(output / "baseline_test_prediction_df.pkl")
    metrics_baselines_test_df = pd.read_pickle(output / "baseline_test_metrics_df.pkl")
    agg_metrics_baselines_test_df = pd.read_pickle(
        output / "baseline_test_aggregate_metrics.pkl"
    )


    pred_val_df = pd.read_pickle(output / "ml_single_step_prediction_val_df.pkl")
    metrics_val_df = pd.read_pickle(output / "ml_single_step_metrics_val_df.pkl")
    pred_auto_stat_val_df = pd.read_pickle(
        output / "ml_single_step_prediction_auto_stationary_val_df.pkl"
    )
    metrics_auto_stat_val_df = pd.read_pickle(
        output / "ml_single_step_metrics_auto_stationary_val_df.pkl"
    )
    agg_metrics_auto_stat_val_df = pd.read_pickle(
        output / "ml_single_step_aggregate_metrics_auto_stationary_val.pkl"
    )
    pred_baselines_val_df = pd.read_pickle(output / "baseline_val_prediction_df.pkl")
    metrics_baselines_val_df = pd.read_pickle(output / "baseline_val_metrics_df.pkl")
    agg_metrics_baselines_val_df = pd.read_pickle(
        output / "baseline_val_aggregate_metrics.pkl"
    )
except FileNotFoundError:
    display(HTML("""
    <div class="alert alert-block alert-warning">
    <b>Warning!</b> File not found. Please make sure you have run all notebooks in Chapter08 and 02-Baseline Forecasts using NIXTLA.ipynb in Chapter04
    </div>
    """))

In [ ]:
pred_val_df

In [ ]:
pred_val_df.head(2)

In [ ]:
pred_auto_stat_val_df.head(2)

In [ ]:
help(pd.melt)

In [ ]:
pred_baselines_val_df = pred_baselines_val_df.set_index('timestamp').melt(id_vars = ['LCLid', 'energy_consumption'],
                                                                          value_vars=['AutoETS', 'TBATS'],
                                                                          var_name='Algorithm',
                                                                          value_name='predictions',
                                                                          ignore_index=False)


pred_baselines_test_df = pred_baselines_test_df.set_index('timestamp').melt(id_vars = ['LCLid','energy_consumption'],
                                                                            value_vars=['AutoETS','TBATS'],
                                                                            var_name='Algorithm',
                                                                            value_name='predictions',
                                                                            ignore_index=False)

In [ ]:
pred_baselines_test_df

In [ ]:
pred_val_df = pd.concat([pred_val_df, pred_auto_stat_val_df, pred_baselines_val_df])
pred_val_df.index.name = "timestamp"

pred_wide_val = pd.pivot(
    pred_val_df.reset_index(),
    index=["LCLid", "timestamp"],
    columns="Algorithm",
    values="predictions",
)
pred_wide_val = pred_wide_val.join(
    pred_val_df.loc[
        pred_val_df.Algorithm == "Lasso Regression", ["LCLid", "energy_consumption"]
    ]
    .reset_index()
    .set_index(["LCLid", "timestamp"])
)
pred_wide_val.head()

In [ ]:
pred_test_df = pd.concat([pred_test_df, pred_auto_stat_test_df, pred_baselines_test_df])
pred_test_df.index.name = "timestamp"

pred_wide_test = pd.pivot(
    pred_test_df.reset_index(),
    index=["LCLid", "timestamp"],
    columns="Algorithm",
    values="predictions",
)
pred_wide_test = pred_wide_test.join(
    pred_test_df.loc[
        pred_test_df.Algorithm == "Lasso Regression", ["LCLid", "energy_consumption"]
    ]
    .reset_index()
    .set_index(["LCLid", "timestamp"])
)
pred_wide_test.head()

In [ ]:
metrics_combined_df = pd.concat([metrics_val_df, metrics_auto_stat_val_df])
metrics_combined_df = pd.pivot(
    metrics_combined_df, index="LCLid", columns="Algorithm", values="MAE"
)
metrics_combined_df.head()